# AoF Replication 2 Solutions

Due date: <b>Febuary 27, 2023 Monday 2pm </b>.

This content is authored by Maria Boutchkova for use in the University of Edinbugh Business School Applications of Econometrics course in Spring 2023.

This notebook covers:

* Histograms
* Common support/overlap
* 1-to-n matching by hand

The first computational cell below (with In [ ] in front) contains the solution. Go over the command lines, make sure they make sense to you, click inside the cell, it should become surrounded by a green rectangle, press Esc - the rectangle will become blue, now press Shift+Enter - this will execute the cell and produce the results beneath it.
While a cell is being executed it shows an asterisk in front of "In [ ]".

To remove all output in the notebook and start again, go to the Kernel tab above, select Restart and Clear Output.

You can add command cells for auxiliary commands and/or steps by choosing Insert Cell above or Cell below from the top Menu.

Your work is marked based on the input data provided with the notebook and the results may differ slightly from the ones reported in the paper.

### Task 1: Figure 1

Task 1.1: The file fastfood.csv is part of this current replication 2 test folder. In the cell below issue the required command to read in the data into Stata.

In [ ]:
*** BEGIN SOLUTION
import delimited fastfood.csv, varn(1) clear
*** END SOLUTION

In [ ]:
%browse

Task 1.2: Without recoding any data produce a historgram of wage_st by state with barwidth of 0.03 and have Pennsylvania be plotted first in transparent bars and New Jersey second in grey bars. Save that grpah as pre.gph

In [ ]:
*** BEGIN SOLUTION
twoway (hist wage_st if state==0 , width(0.03) fcolor(none) lcolor(black)) (hist wage_st if state==1 , width(0.03) fcolor(gs7)), legend(order(1 "Pennsylvania" 2 "New Jersey") size(small) ) scheme(sj) saving(pre.gph, replace)
*** END SOLUTION

Task 1.3: Use the same command as in task 1.2 but on wage_st2 and save that grpah as post.gph

In [ ]:
*** BEGIN SOLUTION
twoway (hist wage_st2 if state==0 , width(0.03) fcolor(none) lcolor(black)) (hist wage_st if state==1 , width(0.03) fcolor(gs7)), legend(order(1 "Pennsylvania" 2 "New Jersey") size(small) ) scheme(sj) saving(post.gph, replace)
*** END SOLUTION

### Task 2: Overlap

Task 2.1: Run the logit regression that produces propensity scores - the variable treated already exists under the name state. Compared to the covariates included in the do file card_krueger_overlap you feel that the variable open is redundant (make sure you know why) and the variable nregs11 should have been included. Name the new variable propScore.

In [ ]:
*** BEGIN SOLUTION
local cov2 i.chain i.co_owned ncalls empft emppt nmgrs wage_st inctime firstinc i.bonus pctaff i.meals hrsopen psoda pfry pentree nregs nregs11
logit state `cov2'
predict propScore
*** END SOLUTION

Task 2.2 Produce the histogram of the propensity scores for treated and controls. Save the graph as a Stata gph file named hist_prop_score2 to be able to download to your own machine, examine clearly and determine the areas without common support/overlap.

In [ ]:
*** BEGIN SOLUTION
qui summarize propScore if state & e(sample) 
local treat_total = r(N)
qui summarize propScore if !state & e(sample) 
local control_total = r(N)
twoway (hist propScore if state==1 , width(0.01) fcolor(gs7) freq) (hist propScore if state==0 , width(0.01) fcolor(none) lcolor(black) freq), legend(order(1 "Treated" 2 "Controls") size(small) ) scheme(sj) xtitle("Propensity Score", size(small)) xlabel(0.05(0.05)1, labsize(vsmall)) ytitle("Number of Stores", size(small)) ylabel(, labsize(vsmall)) note("Treated `treat_total'; Potential controls `control_total'")
graph save hist_prop_score2, replace 
*** END SOLUTION

### Task 3: Matching

Task 3.1 Save the observations with propensity scores, together with the restaurant identifier variable sheet and the treated identifier state in a new file called prop_scores.

In [ ]:
*** BEGIN SOLUTION
keep if propScore <.
keep sheet state propScore
save prop_scores, replace
*** END SOLUTION

Task 3.2 Prepare and save the 3 temp files (temp, temp0 and temp1) to be able to perform 1 to 1 matching by hand using the propensity scores saved above without applying any tail cuttoffs. Rename sheet as id.

In [ ]:
*** BEGIN SOLUTION
rename sheet id
sort state
count
save temp, replace
keep if state == 0
drop state
rename id id0
rename propScore propScore0 
count
save temp0, replace
use temp, clear
keep if state == 1
drop state
rename id id1
rename propScore propScore1
count
save temp1, replace
*** END SOLUTION

Task 3.3: Perform 1 to 1 matching by starting with the controls (since they are fewer) and keeping the best treated match for each control observation. Name the absolute value of the difference between the propensity scores of every matched pair diffprop.

In [ ]:
*** BEGIN SOLUTION
use temp0, clear
cross using temp1
gen diffprop = abs(propScore1 - propScore0)
qui bys id0 (diffprop): keep if _n == 1
*** END SOLUTION

Task 3.4 Reshape the matched pairs into long form, rename id back to sheet and save the 1 to 1 matches without duplicates in a file named psmatch_1to1.

In [ ]:
*** BEGIN SOLUTION
egen idmatch = group(id1 id0)
reshape long id propScore, i(idmatch) j(treat)
duplicates drop id, force
rename id sheet
save psmatch_1to1, replace
count
*** END SOLUTION

Task 3.5 Repeat the matching and this time save the 1 to 3 matches without duplicates in a file named psmatch_1to3.

In [ ]:
*** BEGIN SOLUTION
use temp0, clear
cross using temp1
gen diffprop = abs(propScore1 - propScore0)
qui bys id0 (diffprop): keep if _n < 4
egen idmatch = group(id1 id0)
reshape long id propScore, i(idmatch) j(treat)
duplicates drop id, force
rename id sheet
save psmatch_1to3, replace
count
*** END SOLUTION

### Task 4. Table 4

Task 4.1: Starting with the original data, merge the 1to1 matches and perform the two regressions reported in Table 4 and store their results in preparation for using esttab later.

In [ ]:
*** BEGIN SOLUTION
ssc install estout, replace
import delimited fastfood.csv, varn(1) clear
merge m:1 sheet using psmatch_1to1, nogen keep(match)
local cvars southj centralj i.chain co_owned
eststo clear
eststo: reg dfte state
eststo: reg dfte state `cvars'
*** END SOLUTION

Task 4.1: Starting with the original data, merge the 1to3 matches and perform the two regressions reported in Table 4 and store their results in preparation for using esttab later. Do not use eststo clear at this step!

In [ ]:
*** BEGIN SOLUTION
import delimited fastfood.csv, varn(1) clear
merge m:1 sheet using psmatch_1to3, nogen keep(match)
local cvars southj centralj i.chain co_owned
eststo: reg dfte state
eststo: reg dfte state `cvars'
*** END SOLUTION

Task 4.3 Report the 4 regressions you ran in the previous 2 tasks using esttab in the Stata results window. Make sure you can discuss what may be driving the difference in results. 

In [ ]:
*** BEGIN SOLUTION
esttab 
*** END SOLUTION